<a href="https://colab.research.google.com/github/klotardesignstudio/tradebao-cripto-poc/blob/main/04_backtest_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# ==============================================================================
# TRADEBAO POC - NOTEBOOK 04: BACKTEST DE PORTFÓLIO (V3 - COM ANÁLISE DE TRADES)
# ==============================================================================

# 1. IMPORTS E FUNÇÃO RSI
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def calculate_rsi(data, period=14):
    close_delta = data['Close'].diff()
    gain = close_delta.clip(lower=0)
    loss = -1 * close_delta.clip(upper=0)
    avg_gain = gain.ewm(com=period - 1, min_periods=period).mean()
    avg_loss = loss.ewm(com=period - 1, min_periods=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    data['RSI'] = rsi
    return data

# ==============================================================================
# 2. NOVO MOTOR DE BACKTEST UNIFICADO E DETALHADO
# ==============================================================================

def backtest_engine(df_original, capital_inicial, strategy_type, taxa_transacao=0.001):
    """
    Motor de backtest genérico que agora retorna o capital final E um log detalhado de trades.
    """
    df = df_original.copy()

    # --- Lógica de Sinais (depende da estratégia) ---
    if strategy_type == 'Tendencia':
        ema_curta, ema_longa, sma_tendencia = 21, 55, 100
        # hipotese 2 - ema_curta, ema_longa, sma_tendencia = 12, 26, 50
        df['EMA_curta'] = df['Close'].ewm(span=ema_curta, adjust=False).mean()
        df['EMA_longa'] = df['Close'].ewm(span=ema_longa, adjust=False).mean()
        df['SMA_tendencia'] = df['Close'].rolling(window=sma_tendencia).mean()
        df.dropna(inplace=True)
        filtro_tendencia = df['Close'] > df['SMA_tendencia']
        df['sinal_compra'] = (df['EMA_curta'] > df['EMA_longa']) & (df['EMA_curta'].shift(1) < df['EMA_longa'].shift(1)) & filtro_tendencia
        df['sinal_venda'] = (df['EMA_curta'] < df['EMA_longa']) & (df['EMA_curta'].shift(1) > df['EMA_longa'].shift(1))

    elif strategy_type == 'RSI':
        # hipoteste 1 rsi_periodo, rsi_sobrevenda, rsi_saida = 14, 30, 65
        rsi_periodo, rsi_sobrevenda, rsi_saida = 14, 30, 50
        df = calculate_rsi(df, period=rsi_periodo)
        df.dropna(inplace=True)
        df['sinal_compra'] = (df['RSI'].shift(1) > rsi_sobrevenda) & (df['RSI'] <= rsi_sobrevenda)
        df['sinal_venda'] = (df['RSI'].shift(1) < rsi_saida) & (df['RSI'] >= rsi_saida)

    # --- Loop de Backtest com Log Detalhado ---
    capital_btc = capital_inicial
    em_posicao = False
    trades_log = []
    entry_data = {}

    for i, row in df.iterrows():
        if row['sinal_compra'] and not em_posicao:
            em_posicao = True
            entry_data = {'data_entrada': i, 'preco_entrada': row['Close']}
            # Deduz a taxa do capital em BTC antes de 'comprar' a altcoin
            capital_btc *= (1 - taxa_transacao)

        elif row['sinal_venda'] and em_posicao:
            em_posicao = False
            # Deduz a taxa do capital em BTC após 'vender' a altcoin
            capital_btc *= (1 - taxa_transacao)

            duracao = i - entry_data['data_entrada']
            resultado_pct = ((row['Close'] - entry_data['preco_entrada']) / entry_data['preco_entrada']) * 100

            # Adiciona o resultado líquido (já descontando as duas taxas no capital)
            trades_log.append({**entry_data, 'data_saida': i, 'preco_saida': row['Close'], 'duracao': duracao, 'resultado_pct': resultado_pct})

            # Atualiza o capital
            capital_btc *= (1 + (resultado_pct / 100))
            entry_data = {}

    # Se o backtest terminar com uma posição aberta, não a consideramos no log final de trades fechados,
    # mas ajustamos o capital final com base no último preço (incluindo taxa de venda).
    if em_posicao:
        last_price = df['Close'].iloc[-1]
        final_trade_pct = ((last_price - entry_data['preco_entrada']) / entry_data['preco_entrada']) * 100
        capital_btc *= (1 + (final_trade_pct / 100))
        capital_btc *= (1 - taxa_transacao)

    return capital_btc, pd.DataFrame(trades_log)

# ==============================================================================
# 3. CONFIGURAÇÕES E EXECUÇÃO DO PORTFÓLIO
# ==============================================================================

simbolos = ["fetbtc", "rndrbtc", "agixbtc", "dogebtc", "ftmbtc", "arbusd", "injbtc", "maticbtc", "sushibtc", "imxbtc", "sandbtc", "galabtc", "arbtc", "filbtc", "ombtc"]
capital_total_inicial = 1.0
alocacao_tendencia = 0.3
alocacao_rsi = 0.7
# hipotese 3
#alocacao_tendencia = 0.9
#alocacao_rsi = 0.1

# Listas para coletar os logs de todos os trades
log_total_tendencia = []
log_total_rsi = []
resultados_portfolio = [] # Para o resumo de rentabilidade

for simbolo in simbolos:
    caminho_do_arquivo = f'/content/drive/MyDrive/Colab Notebooks/TradeBaoCripto/dados_binance_{simbolo}_1h.csv'

    try:
        df_ativo = pd.read_csv(caminho_do_arquivo, index_col='Open Time', parse_dates=True)
        print(f"----- Processando Portfólio para {simbolo.upper()} -----")
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado para {simbolo}. Pulando.")
        continue

    # Rodar os backtests e coletar os logs
    resultado_final_tendencia, trades_tendencia = backtest_engine(df_ativo, capital_total_inicial * alocacao_tendencia, 'Tendencia')
    resultado_final_rsi, trades_rsi = backtest_engine(df_ativo, capital_total_inicial * alocacao_rsi, 'RSI')

    if not trades_tendencia.empty:
        log_total_tendencia.append(trades_tendencia)
    if not trades_rsi.empty:
        log_total_rsi.append(trades_rsi)

    # O resto do cálculo de rentabilidade continua como antes
    capital_final_portfolio = resultado_final_tendencia + resultado_final_rsi
    rent_tendencia = ((resultado_final_tendencia - (capital_total_inicial * alocacao_tendencia)) / (capital_total_inicial * alocacao_tendencia)) * 100
    rent_rsi = ((resultado_final_rsi - (capital_total_inicial * alocacao_rsi)) / (capital_total_inicial * alocacao_rsi)) * 100
    rent_portfolio = ((capital_final_portfolio - capital_total_inicial) / capital_total_inicial) * 100
    rent_bh = ((df_ativo['Close'].iloc[-1] - df_ativo['Close'].iloc[0]) / df_ativo['Close'].iloc[0]) * 100 if not df_ativo.empty else 0
    resultados_portfolio.append({'Ativo': simbolo, 'Rent. Tendência (%)': rent_tendencia, 'Rent. RSI (%)': rent_rsi, 'Rent. PORTFÓLIO (%)': rent_portfolio, 'Rent. B&H (%)': rent_bh})

# ==============================================================================
# 4. ANÁLISE DE PERFORMANCE E COMPORTAMENTO
# ==============================================================================

print("\n\n===== RESUMO CONSOLIDADO DO PORTFÓLIO HÍBRIDO =====")
resumo_df = pd.DataFrame(resultados_portfolio)
print(resumo_df.round(2).to_string(index=False))

if log_total_tendencia:
    df_tendencia_trades = pd.concat(log_total_tendencia)
    print("\n\n===== ANÁLISE DE COMPORTAMENTO - ESTRATÉGIA DE TENDÊNCIA =====")
    print(f"Número Total de Trades: {len(df_tendencia_trades)}")
    print(f"Duração Média de um Trade: {df_tendencia_trades['duracao'].mean()}")
    print(f"Duração Máxima de um Trade: {df_tendencia_trades['duracao'].max()}")
    print(f"Taxa de Acerto (Win Rate): {((df_tendencia_trades['resultado_pct'] > 0).sum() / len(df_tendencia_trades) * 100):.2f}%")
    print(f"Lucro Médio (nos trades vencedores): {df_tendencia_trades[df_tendencia_trades['resultado_pct'] > 0]['resultado_pct'].mean():.2f}%")
    print(f"Prejuízo Médio (nos trades perdedores): {df_tendencia_trades[df_tendencia_trades['resultado_pct'] < 0]['resultado_pct'].mean():.2f}%")

if log_total_rsi:
    df_rsi_trades = pd.concat(log_total_rsi)
    print("\n\n===== ANÁLISE DE COMPORTAMENTO - ESTRATÉGIA RSI =====")
    print(f"Número Total de Trades: {len(df_rsi_trades)}")
    print(f"Duração Média de um Trade: {df_rsi_trades['duracao'].mean()}")
    print(f"Duração Máxima de um Trade: {df_rsi_trades['duracao'].max()}")
    print(f"Taxa de Acerto (Win Rate): {((df_rsi_trades['resultado_pct'] > 0).sum() / len(df_rsi_trades) * 100):.2f}%")
    print(f"Lucro Médio (nos trades vencedores): {df_rsi_trades[df_rsi_trades['resultado_pct'] > 0]['resultado_pct'].mean():.2f}%")
    print(f"Prejuízo Médio (nos trades perdedores): {df_rsi_trades[df_rsi_trades['resultado_pct'] < 0]['resultado_pct'].mean():.2f}%")

----- Processando Portfólio para FETBTC -----
----- Processando Portfólio para RNDRBTC -----
----- Processando Portfólio para AGIXBTC -----
----- Processando Portfólio para DOGEBTC -----
----- Processando Portfólio para FTMBTC -----
----- Processando Portfólio para ARBUSD -----
----- Processando Portfólio para INJBTC -----
----- Processando Portfólio para MATICBTC -----
----- Processando Portfólio para SUSHIBTC -----
----- Processando Portfólio para IMXBTC -----
----- Processando Portfólio para SANDBTC -----
----- Processando Portfólio para GALABTC -----
----- Processando Portfólio para ARBTC -----
----- Processando Portfólio para FILBTC -----
----- Processando Portfólio para OMBTC -----


===== RESUMO CONSOLIDADO DO PORTFÓLIO HÍBRIDO =====
   Ativo  Rent. Tendência (%)  Rent. RSI (%)  Rent. PORTFÓLIO (%)  Rent. B&H (%)
  fetbtc                 7.31         -18.61               -10.84           3.78
 rndrbtc                92.92           7.15                32.88         324.69
 agixb